In [ ]:
import cobra
import itertools
import pandas as pd
from openpyxl import Workbook
inputdir = "/Users/liuyushuo/Desktop/结果汇总/model/"    #模型存储文件夹
outputdir = "/Users/liuyushuo/Desktop/结果汇总/result/output(ACCOA)1/"    #输出文件夹
model = cobra.io.read_sbml_model(inputdir+"Metacyc6578.xml") #模型名字
# steps = 20    #限定步数（含有交换反应，所以不准）
Cfixationslen = 48    #前多少个反应进行组合
a=range(0,Cfixationslen)

reactions_data = [
    ('R_EX_56', '0.5 ATP_c + 0.5 WATER_c --> 0.5 ADP_c + 0.5 Pi_c', (-1000, 1000), 0),
    ('R_EX_57', '0.25 ATP_c + 0.25 WATER_c --> 0.25 AMP_c + 0.25 PPI_c', (-1000, 1000), 0),
    ('R_EX_89', 'NADPH_c --> NADP_c + 2.0 PROTON_c', (-1000, 1000), 0),
    ('R_EX_90', 'NADH_c --> NAD_c + 2.0 PROTON_c', (-1000, 1000), 0),
    ('R_EX_901', 'NADH_c + Acceptor_c + PROTON_c --> NAD_c + Donor__45__H2_c', (-1000, 1000), 0),
    ('R_EX_902', 'NADH_c + Ox__45__Thioredoxin_c --> NAD_c + Red__45__Thioredoxin_c', (-1000, 1000), 0),
    ('R_EX_903', 'NADH_c + Oxidized__45__Amicyanins_c --> NAD_c + Reduced__45__Amicyanins_c', (-1000, 1000), 0),
    ('R_EX_904', 'NADH_c + ETF__45__Oxidized_c --> NAD_c + ETF__45__Reduced_c', (-1000, 1000), 0),
    # ('objPYR_c', 'PYRUVATE_c --> ', (3.33333, 3.33333), 1)
    # ('objG3P_c', 'G3P_c --> ', (3.33333, 3.33333), 1)
    # ('objGLYOX_c', 'GLYOX_c --> ', (5, 5), 1)
    ('objACCOA_c', 'ACETYL__45__COA_c --> CO__45__A_c', (5, 5), 1)
]

for reaction_name, reaction_equation, bounds, object in reactions_data:
    reaction = cobra.Reaction(reaction_name)
    model.add_reaction(reaction)
    reaction.build_reaction_from_string(reaction_equation)
    reaction.lower_bound, reaction.upper_bound = bounds
    reaction.objective_coefficient = object

model.objective = 'objACCOA_c'
model_obj = 'objACCOA_c'

#为了得到最少反应步数，权重要加上
下边语句是给前71个反应添加权重，循环前71个反应，model.reactions[i].get_coefficient(j)获取代谢物系数后乘以4，然后放到字典dict1{}，dict1={}就变成:{<Metabolite ERYTHROSE__45__4P_c at 0x7efeab2dacf8>: -4.0, <Metabolite GLYCOLALDEHYDE_c at 0x7efeab2da160>: -4.0, <Metabolite D__45__ALLOSE__45__6__45__PHOSPHATE_c at 0x7efeab2da0b8>: 4.0}.
然后把dict1再添加到代谢物列表中，原来的代谢物不变，而系数都加了4，原来-1的变成-5，原来1的变成5

In [ ]:
# for i in a:
#     dict1={}
#     for j in model.reactions[i].metabolites:
#         dict1[j] = model.reactions[i].get_coefficient(j) * (4)
#     model.reactions[i].add_metabolites(dict1)

# for i in model.reactions:
#     if i.id.startswith("R_EX_") and i.id != "R_EX_010":  # 判断反应ID是否以R_EX_开头且不等于R_EX_010
#         dict1 = {}
#         for j in i.metabolites:
#             dict1[j] = i.get_coefficient(j) * (99)  # 设置权重为系数的100倍
#         i.add_metabolites(dict1)  # 将修改后的代谢物及其权重添加到反应中

In [ ]:
def output_txt(need_fluxes,outfile):
    with open(outfile,'w') as outf:
        for need_id in need_fluxes.index: #need_fluxes.index有通量的反应的名字
            rea = model.reactions.get_by_id(need_id) #得到相应反应的反应方程
            need_fluxes[need_id] #包含内容为objACET_c   5
            fl = str(round(need_fluxes[need_id],5)) #  round(x,n) 方法返回浮点数x保留n位小数的值 
            outline_list = [need_id,fl,rea.reaction,str(rea.lower_bound),str(rea.upper_bound),str(rea.objective_coefficient)]
            outf.write("\t".join(outline_list)+'\n')#  sep.join(seq) 其中sep：分隔符。可以为空。seq：要连接的元素序列、字符串、元组、字典上面的语法即：以sep作为分隔符，将seq所有的元素合并成一个新的字符串

In [ ]:
def output_excel(need_fluxes, excel_file):
    # 将结果输出到Excel文件中
    data = []
    for need_id in need_fluxes.index:
        rea = model.reactions.get_by_id(need_id)
        fl = round(need_fluxes[need_id], 5)
        # 提取 annotation 中的 ec_number 和 kegg_id
        ec_number = rea.annotation.get('ec_number', '')  # 如果没有 ec_number，则默认为空字符串
        kegg_id = rea.annotation.get('kegg_id', '')      # 如果没有 kegg_id，则默认为空字符串
        data.append([need_id, fl, rea.reaction, rea.lower_bound, rea.upper_bound, rea.objective_coefficient, ec_number, kegg_id])
    
    # 创建 DataFrame
    df = pd.DataFrame(data, columns=['Reaction ID', 'Flux', 'Reaction Equation', 'Lower Bound', 'Upper Bound', 'Objective Coefficient', 'EC Number', 'KEGG id'])
    
    # 保存到 Excel 文件
    df.to_excel(excel_file, index=False)

In [ ]:
# 关闭其余的HCO3的固碳反应
model.reactions.get_by_id('RXN-12893').bounds=(0,0)
model.reactions.get_by_id('R524-RXN').bounds=(0,0)
model.reactions.get_by_id('RXN-16909').bounds=(0,0)
model.reactions.get_by_id('RXN-13202').bounds=(0,0)
model.reactions.get_by_id('CARBPSYN-RXN').bounds=(0,0)
model.reactions.get_by_id('BIOTIN-CARBOXYL-RXN').bounds=(0,0)

# 关闭其余的CO2的固碳反应
model.reactions.get_by_id('3.5.3.21-RXN').bounds=(0,0)
model.reactions.get_by_id('RXN-13926-CPD-8619/NADP//CPD-8620/CARBON-DIOXIDE/NADPH.45.').bounds=(0,0)
model.reactions.get_by_id('RXN-13926-CPD-8619/NAD//CPD-8620/CARBON-DIOXIDE/NADH.43.').bounds=(0,0)
model.reactions.get_by_id('RXN-13926-CPD-4702/NADP//CPD-4581/CARBON-DIOXIDE/NADPH.45.').bounds=(0,0)
model.reactions.get_by_id('RXN-13926-CPD-4702/NAD//CPD-4581/CARBON-DIOXIDE/NADH.43.').bounds=(0,0)
model.reactions.get_by_id('RXN-13926-CPD-4577/NADP//CPD-4578/CARBON-DIOXIDE/NADPH.45.').bounds=(0,0)
model.reactions.get_by_id('RXN-13926-CPD-4577/NAD//CPD-4578/CARBON-DIOXIDE/NADH.43.').bounds=(0,0)
model.reactions.get_by_id('PREPHENATE-DEHYDROGENASE-NADP+-RXN').bounds=(0,0)
model.reactions.get_by_id('RXN-15677').bounds=(0,0)
model.reactions.get_by_id('RXN-14638').bounds=(0,0)
model.reactions.get_by_id('ARGDECARBOX-RXN[CCO-CYTOSOL]-ARG/PROTON//CARBON-DIOXIDE/AGMATHINE.50.').bounds=(0,0)
model.reactions.get_by_id('4-HYDROXYBENZOATE-DECARBOXYLASE-RXN').bounds=(0,0)
model.reactions.get_by_id('RXN-802').bounds=(0,0)
model.reactions.get_by_id('1.3.1.53-RXN').bounds=(0,0)
model.reactions.get_by_id('PHENGLYOXRED-RXN').bounds=(0,0)
model.reactions.get_by_id('CYSTAUR-RXN').bounds=(0,0)
model.reactions.get_by_id('RXN-14466').bounds=(0,0)
model.reactions.get_by_id('RXN-16008').bounds=(0,0)
model.reactions.get_by_id('RXN-11736').bounds=(0,0)
model.reactions.get_by_id('CARBOXY-OXOHEPT-ENEDIOATE-DECARBOXY-RXN').bounds=(0,0)
model.reactions.get_by_id('PROTOCATECHUATE-DECARBOXYLASE-RXN').bounds=(0,0)
model.reactions.get_by_id('RXN-15580').bounds=(0,0)
model.reactions.get_by_id('RXN-803').bounds=(0,0)
model.reactions.get_by_id('CARBAMATE-KINASE-RXN').bounds=(0,0)
model.reactions.get_by_id('1.1.1.170-RXN-CPD-5846/NAD//4-ALPHA-METHYL-5-ALPHA-CHOLEST-7-EN-3-ON/CARBON-DIOXIDE/NADH.75.').bounds=(0,0)
model.reactions.get_by_id('1.3.1.67-RXN-CIS-12-DIHYDROXY-ETCETERA-CARBOXYLATE/NAD//CARBON-DIOXIDE/4-METHYLCATECHOL/NADH.80.').bounds=(0,0)
model.reactions.get_by_id('1.2.7.4-RXN').bounds=(0,0)
model.reactions.get_by_id('RXN-12907').bounds=(0,0)
model.reactions.get_by_id('KETOISOCAPROATE-RXN').bounds=(0,0)


# # 关闭甲酸参与的反应
# model.reactions.get_by_id('FORMALDEHYDE-DEHYDROGENASE-RXN').bounds=(0,0)
# model.reactions.get_by_id('GARTRANSFORMYL2-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN-10066').bounds=(0,0)
# model.reactions.get_by_id('RXN0-1382').bounds=(0,0)
# model.reactions.get_by_id('RXN-17561').bounds=(0,0)
# model.reactions.get_by_id('PYRUVFORMLY-RXN').bounds=(0,0)
# model.reactions.get_by_id('FORMATETHFLIG-RXN-THF/ATP/FORMATE//10-FORMYL-THF/ADP/Pi.38.').bounds=(0,0)
# model.reactions.get_by_id('LACTATE-ALDOLASE-RXN').bounds=(0,0)
# model.reactions.get_by_id('FORMATE--DIHYDROFOLATE-LIGASE-RXN-DIHYDROFOLATE/FORMATE/ATP//CPD-14932/ADP/Pi.44.').bounds=(0,0)
# model.reactions.get_by_id('FORMATETHFLIG-RXN-CPD-1301/ATP/FORMATE//CPD-17116/ADP/Pi.39.').bounds=(0,0)
# model.reactions.get_by_id('FORMATE-DEHYDROGENASE-NADP+-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN0-3281').bounds=(0,0)
# model.reactions.get_by_id('RXN-12274').bounds=(0,0)
# model.reactions.get_by_id('1.2.1.2-RXN').bounds=(0,0)
# model.reactions.get_by_id('FHLMULTI-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN-11453').bounds=(0,0)
# model.reactions.get_by_id('RXN-17615').bounds=(0,0)
# model.reactions.get_by_id('OXALATE-DECARBOXYLASE-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN-14028-CPD-17235/WATER//CPD-17182/FORMATE.35.').bounds=(0,0)
# model.reactions.get_by_id('RXN-14028-CPD-17234/WATER//methanofuran-d/FORMATE.40.').bounds=(0,0)
# model.reactions.get_by_id('RXN-14028-CPD-17233/WATER//CPD-7629/FORMATE.34.').bounds=(0,0)
# model.reactions.get_by_id('RXN-14028-CPD-17232/WATER//CPD-7628/FORMATE.34.').bounds=(0,0)
# model.reactions.get_by_id('RXN-15187').bounds=(0,0)
# model.reactions.get_by_id('GTP-CYCLOHYDRO-I-RXN').bounds=(0,0)
# model.reactions.get_by_id('FORMYLTHFDEFORMYL-RXN-CPD-17116/WATER//CPD-1301/FORMATE/PROTON.41.').bounds=(0,0)
# model.reactions.get_by_id('FORMYLTHFDEFORMYL-RXN-10-FORMYL-THF/WATER//THF/FORMATE/PROTON.40.').bounds=(0,0)
# model.reactions.get_by_id('KETOBUTFORMLY-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN-7593').bounds=(0,0)
# model.reactions.get_by_id('FORMYL-COA-HYDROLASE-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN-2884').bounds=(0,0)
# model.reactions.get_by_id('RXN1G-129').bounds=(0,0)
# model.reactions.get_by_id('3.7.1.9-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN-10824').bounds=(0,0)
# model.reactions.get_by_id('RXN-10063').bounds=(0,0)
# model.reactions.get_by_id('ARYLFORMAMIDASE-RXN').bounds=(0,0)
# model.reactions.get_by_id('N-FORMYLGLUTAMATE-DEFORMYLASE-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN-7586').bounds=(0,0)
# model.reactions.get_by_id('1.14.13.70-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN-10056').bounds=(0,0)
# model.reactions.get_by_id('RXN-11054').bounds=(0,0)
# model.reactions.get_by_id('DIOHBUTANONEPSYN-RXN').bounds=(0,0)
# model.reactions.get_by_id('GTP-CYCLOHYDRO-II-RXN').bounds=(0,0)
# model.reactions.get_by_id('RXN-11318').bounds=(0,0)
# model.reactions.get_by_id('FORMAMIDASE-RXN').bounds=(0,0)

In [ ]:
closeidno1 = itertools.combinations(range(1, Cfixationslen+1), 1)    #组合：前Cfixationslen中选 1 个反应打开

# 创建一个新的 Excel 工作簿
wb = Workbook()
ws = wb.active

# 写入表头
ws.append(["Combination", "Target", "Reaction Number", "ATP Number", "NADPH Number", "NADH Number"])

for i in closeidno1:
    # model.reactions[i[0]-1].bounds = (-1000.0,1000.0)    #逐个打开前48个反应
    model.reactions[i[0]-1].bounds = (0, 10)    #逐个打开前48个反应#
    # model.reactions[i[0]-1].bounds = (0, 0) 
    try:
        pfba_solution = cobra.flux_analysis.pfba(model)
        need_fluxes =pfba_solution.fluxes[abs(pfba_solution.fluxes)>1e-10]

        # 筛选出不以 obj 或 R_EX_ 开头的反应
        # filtered_fluxes = {reaction_id: flux for reaction_id, flux in need_fluxes.items()
        #                    if not (reaction_id.startswith('obj') or reaction_id.startswith('R_EX_') or reaction_id.startswith('RXN0-5224'))}
        # print(len(filtered_fluxes))

        filtered_fluxes = {reaction_id: flux for reaction_id, flux in need_fluxes.items()
                        if not (reaction_id.startswith('obj') or 
                                reaction_id.startswith('R_EX_') or 
                                reaction_id.startswith('RXN0-5224') or
                                reaction_id.startswith('RXN-12957') or
                                reaction_id.startswith('GLUTARYL-COA-DEHYDROG-RXN') or
                                reaction_id.startswith('PROPANOATECOA-LIGASE-RXN') or
                                reaction_id.startswith('ACETATE--COA-LIGASE-ADP-FORMING-RXN') or
                                reaction_id.startswith('RXN-16774') or
                                reaction_id.startswith('RXN-16767') or
                                reaction_id.startswith('RXN-14204') or
                                reaction_id.startswith('GLUCONATE-2-DEHYDROGENASE-RXN'))}

        # 检查固碳反应的通量是否都不为零
        reaction1_flux = pfba_solution.fluxes.get(model.reactions[i[0] - 1].id, 0)
        print(reaction1_flux)

        if abs(reaction1_flux) < 1e-10:
            print(f"反应 {i} 的通量为零，跳过输出")
        elif len(need_fluxes) == 0:
        # elif len(need_fluxes) == 0 or len(filtered_fluxes) > steps:
            print(f"反应 {i} 的组合没有通量结果，跳过输出")
        else:
            # 输出完整的反应通量到文本文件
            outfile = outputdir + str(i) + '.txt'
            output_txt(need_fluxes, outfile)

            # 输出完整的反应通量到 Excel 文件
            excel_file = outputdir + str(i) + '.xlsx'
            output_excel(need_fluxes, excel_file)

            # 检查需要的通量是否存在
            r_ex_56_flux = need_fluxes.get('R_EX_56', 0)
            r_ex_57_flux = need_fluxes.get('R_EX_57', 0)
            r_ex_89_flux = need_fluxes.get('R_EX_89', 0)
            r_ex_90_flux = need_fluxes.get('R_EX_90', 0)
            
            # 计算ATP组合通量
            combined_flux = (r_ex_56_flux) * (-0.05) + (r_ex_57_flux) * (-0.05)

            # 写入 summary 文件，保留3位小数
            ws.append([str(i), model.reactions.get_by_id(model_obj).reaction, len(filtered_fluxes), f"{combined_flux:.3f}", f"{(r_ex_89_flux) * (-0.1):.3f}", f"{(r_ex_90_flux) * (-0.1):.3f}"])
            print(f"成功处理反应 {i}")

    except Exception as e:
        print(f"反应 {i} 出现错误: {e}")
    
    model.reactions[i[0]-1].bounds = (0.0,0.0)

# 保存 Excel 文件
wb.save(outputdir + "summary.xlsx")